In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
model = keras.models.load_model('./Trained Models/m3.h5')
# url = '/home/avishrant/Downloads/test2.mp4'
#url = './Testing/test2.mp4'

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 128)         1

In [4]:
import cv2
import numpy as np
vid = cv2.VideoCapture(0)
lst = []

font = cv2.FONT_HERSHEY_SIMPLEX 
org = (50, 50) 
fontScale = 1
color = (255, 0, 0) 
thickness = 2
while (cv2.waitKey(1) == -1):
    ret, frame = vid.read()
    if not ret:
        break
    tmp = cv2.resize(frame , (64, 64))
    tmp = tmp / 255.0
    pred = model.predict(np.array([tmp]))
    frame = cv2.putText(frame, str(pred[0][0]), org, font, fontScale, color, thickness, cv2.LINE_AA) 
    cv2.imshow("Video", frame)
    
vid.release()
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np
vid = cv2.VideoCapture(0)
lst = []
while (cv2.waitKey(1) == -1):
    ret, frame = vid.read()
    if not ret:
        break
    lst.append(cv2.resize(frame, (150,150)))

In [5]:
lst2 = np.array(lst)
pred = model.predict(lst2)

In [6]:
output_folder = 'output/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [7]:
count = 0
fcount = 0
vid = cv2.VideoCapture(url)
for i in range(len(pred)):
    ret, frame = vid.read()
    if fcount == i and pred[i] < 0.1:
        face_filename = '%s/%d.jpg' % (output_folder, count)
        cv2.imwrite(face_filename, frame)
        count += 1
    fcount += 1
    